# Agente de Email con Autenticación Dinámica
Este notebook documentado explica la creación de un agente inteligente con capas de seguridad y aprobación humana.

email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

### 1. Configuración de Entorno
Cargamos las variables de entorno para usar Bedrock y las utilidades de visualización.


In [1]:
from dotenv import load_dotenv
from IPython.display import display, Image, Markdown, Latex
load_dotenv()


True

### 2. Contexto del Agente (`EmailContext`)
Definimos los datos estáticos que el agente consultará para validar la identidad del usuario.


In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

### 3. Estado de Autenticación (`AuthenticatedState`)
Extendemos la memoria del agente para incluir el campo `authenticated`, permitiendo rastrear la sesión.


In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

### 4. Herramientas del Agente (Tools)
Definimos las funciones que el agente puede ejecutar: consultar inbox, enviar correos y el validador de identidad.


In [ ]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

### 5. Middleware de Herramientas Dinámicas
Controla qué herramientas están disponibles según el estado de autenticación del usuario.


In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

### 6. Middleware de Prompt Dinámico
Cambia el comportamiento del agente ( System Prompt) basándose en si el acceso ha sido concedido.


In [ ]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt_func(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

### 7. Configuración del Modelo de Lenguaje (LLM)
Configuramos el acceso al modelo de IA a través de Amazon Bedrock.


In [ ]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

In [ ]:
# CREACIÓN DEL AGENTE:
# Aquí se orquestan todos los componentes: 
# El LLM, las herramientas, el esquema de estado, el contexto y los middlewares.
agent = create_agent(
        llm,
        tools=[authenticate, check_inbox, send_email],
        state_schema=AuthenticatedState,
        context_schema=EmailContext,
        middleware=[
            dynamic_tool_call,
            dynamic_prompt_func,
            HumanInTheLoopMiddleware(
                interrupt_on={
                    "authenticate": False,
                    "check_inbox": False,
                    "send_email": True, # Pausa para aprobación humana en esta herramienta
                }
            ),
        ],
    )

### 8. Orquestación del Agente
Unimos todos los componentes anteriores para crear un agente inteligente con seguridad integrada.


In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="draft 1")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

### 9. Prueba de Ejecución
Iniciamos una conversación de prueba para observar el comportamiento del agente y la autenticación.


In [ ]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

### 10. Inspección de Salida
Analizamos la estructura del mensaje generado durante una interrupción de seguridad.


In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

### 11. Reanudación del Proceso (Resume)
Proporcionamos la aprobación humana necesaria para completar el envío del correo pausado.


In [ ]:
from pprint import pprint

pprint(response)

### 12. Depuración Final
Imprimimos el estado completo de la respuesta para verificar que el flujo fue exitoso.
